# Web Accessibility
---

## Video: ¿Qué es la accesibilidad web?

[¿Qué es la accesibilidad web?](https://www.youtube.com/watch?v=gQIKQO41pME)

## a11y

![a11y](img/aw_01_intro_a11y.png)

## Problemas de accesibilidad en el mundo real

![Problemas de accesibilidad en el mundo real](img/aw_01_intro_problemas.png)

## Definición AW

![Definición de accesibilidad web](img/aw_taller_definicion.png)

## Ejemplo

### Lo que estamos viendo

![Lo que estamos viendo](img/aw_03_discapacidad_ejemplo_ven.png)

### Lo que ven las personas ciegas

![Lo que ven las personas ciegas](img/aw_03_discapacidad_ejemplo_noven.png)

### Datos estadísticos

![Discapacidad en el mundo](img/aw_03_discapacidad_estadistica.png)

## Legislación

### Pautas y Leyes a nivel mundial

![W3C](img/aw_04_pautas_y_leyes_w3c.png)

|              |                                                                               |
| --           | --                                                                            |
| **W3C**      | World Wide Web Consortium                                                     |
| **WAI**      | Web Accessibility Initiative                                                  |
| **WCAG 2.0** | Web Content Accessibility Guidelines 2.0 [URL](https://www.w3.org/TR/WCAG20/) |

### WCAG 2.0

![WCAG 2.0](img/aw_04_pautas_y_leyes_wcag2.0_español.png)

### Técnicas

| Criterio 1.1.1 |
| --- |
| ![CAPTCHA](img/aw_05_tecnicas_captcha.png) |

| Criterio 1.2.2 |
| --- |
| ![Video CC](img/aw_05_tecnicas_videoscc.jpeg) |

| Criterio 1.1.1 |
| --- |
| ![Imágenes con atributo ALT](img/aw_05_tecnicas_alt.jpg) |

| Criterio 1.4.2 |
| --- |
| ![Pausar audio](img/aw_05_tecnicas_audio.jpeg) |

| Criterio 1.4.3 |
| --- |
| ![Alto contraste](img/aw_05_tecnicas_contraste.png) |
| ![Ejemplo Alto contraste](img/aw_05_tecnicas_contraste2.png) |
| Ejemplo: Poco contraste vs Mucho contraste |

| Criterio 2.4.2 |
| --- |
| ![Titulos y encabezados](img/aw_05_tecnicas_encabezados.jpg) |

| Criterio 2.4.4 |
| --- |
| ![Textos descriptivos](img/aw_05_tecnicas_texto_descriptivo.png) |

| Criterio 2.1.1 |
| --- |
| ![Mouse + Teclado](img/aw_05_tecnicas_mouse_teclado.jpg) |

| Criterio 2.4.3 |
| --- |
| ![Foco](img/aw_05_tecnicas_foco.jpg) |

| Criterio 2.2.1 |
| --- |
| ![Tiempo suficiente](img/aw_05_tecnicas_tiempo.png) |

| Criterio 2.3.1 |
| --- |
| ![Destellos](img/aw_05_tecnicas_destellos.png) |

| |
| -- |
| ![Palabras simples y párrafos breves](img/aw_05_tecnicas_palabras_simples.png) |

| |
| -- |
| ![Diseño responsivo](img/aw_05_tecnicas_disenio_responsivo.png) |

| Criterio 3.3.1 |
| --- |
| ![Diseño responsivo](img/aw_05_tecnicas_disenio_responsivo2.png) |

### Video: ¿Cómo diseñar un sitio web accesible?

[¿Cómo diseñar un sitio web accesible?](https://www.youtube.com/watch?v=5WrB508ZpNI)

### Legislación en Argentina

![Legislación en Argentina](img/aw_04_pautas_y_leyes_arg.png)
[ONTI](https://www.argentina.gob.ar/jefatura/innovacion-publica/onti/evaluacion-accesibilidad-web)

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install selenium

In [4]:
!pip install validators

In [13]:
class Utils:

    @staticmethod
    def xpath_soup(element):
        # pylint: disable=consider-using-f-string
        """This function calculates the xpath of an element"""
        if element is None:
            return '/html'
        components = []
        child = element if element.name else element.parent
        for parent in child.parents:  # type: bs4.element.Tag
            siblings = parent.find_all(child.name, recursive=False)
            components.append(
                child.name if 1 == len(siblings) else '%s[%d]' % (
                    child.name,
                    next(i for i, s in enumerate(siblings, 1) if s is child)
                    )
                )
            child = parent
        components.reverse()
        if not components:
            return '/html'
        return '/%s' % '/'.join(components)

    @staticmethod
    def extract_texts(soup):
        """This function extracts all texts from a page"""
        soup2 = soup
    
        # remove script, style and title elements
        for invisible_element in soup2(["script", "style", "title", "noscript"]):
            invisible_element.extract()
    
        # remove comments
        comments = soup2.findAll(string=lambda text:isinstance(text, Comment))
        for comment in comments:
            comment.extract()
    
        # remove doctype
        doctype = soup2.find(string=lambda text:isinstance(text, Doctype))
        if not doctype is None:
            doctype.extract()
    
        # get all elements with text
        texts = []
        texts_on_page = soup2.findAll(string=True)
        for text in texts_on_page:
            if not text.strip() == "" and not text == "\n":
                texts.append(text.parent)
    
        return texts

    @staticmethod
    def get_background_color(driver, text):
        """This function returns the background color of a given text"""
        if text is None:
            return "rgba(255,255,255,1)"
    
        selenium_element = driver.find_element(by="xpath", value=Utils.xpath_soup(text))
        background_color = Utils.convert_to_rgba_value(selenium_element.value_of_css_property('background-color'))
    
        if eval(background_color[4:])[3] == 0:
            return Utils.get_background_color(driver, text.parent)
    
        return background_color

    @staticmethod
    def convert_to_rgba_value(color):
        """This function converts a color value to the rgba format"""
        if color[:4] != "rgba":
            rgba_tuple = eval(color[3:]) + (1,)
            color = "rgba" + str(rgba_tuple)
    
        return color

    @staticmethod
    def get_contrast_ratio(text_color, background_color):
        """This function calculates the contrast ratio between text color and background color"""
        # preparing the RGB values
        r_text = Utils.convert_rgb_8bit_value(text_color[0])
        g_text = Utils.convert_rgb_8bit_value(text_color[1])
        b_text = Utils.convert_rgb_8bit_value(text_color[2])
        r_background = Utils.convert_rgb_8bit_value(background_color[0])
        g_background = Utils.convert_rgb_8bit_value(background_color[1])
        b_background = Utils.convert_rgb_8bit_value(background_color[2])
    
        # calculating the relative luminance
        luminance_text = 0.2126 * r_text + 0.7152 * g_text + 0.0722 * b_text
        luminance_background = 0.2126 * r_background + 0.7152 * g_background + 0.0722 * b_background
    
        # check if luminance_text or luminance_background is lighter
        if luminance_text > luminance_background:
            # calculating contrast ration when luminance_text is the relative luminance of the lighter colour
            contrast_ratio = (luminance_text + 0.05) / (luminance_background + 0.05)
        else:
            # calculating contrast ration when luminance_background is the relative luminance of the lighter colour
            contrast_ratio = (luminance_background + 0.05) / (luminance_text + 0.05)
    
        return contrast_ratio

    @staticmethod
    def convert_rgb_8bit_value(single_rgb_8bit_value):
        """This function converts an rgb value to the needed format"""
        # dividing the 8-bit value through 255
        srgb = single_rgb_8bit_value / 255
    
        # check if the srgb value is lower than or equal to 0.03928
        if srgb <= 0.03928:
            return srgb / 12.92
    
        return ((srgb + 0.055) / 1.055) ** 2.4

In [15]:
import sys
import urllib.parse
from bs4 import BeautifulSoup, Comment, Doctype
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions

class AccessibilityTester:
  
    def __init__(self, url):
        self.url = url
        self.required_degree = 0
        self.driver = None
        self.page = None
        self.correct = {"doc_language":0, "alt_texts":0, "input_labels":0, "empty_buttons":0, "empty_links":0, "color_contrast":0}
        self.wrong   = {"doc_language":0, "alt_texts":0, "input_labels":0, "empty_buttons":0, "empty_links":0, "color_contrast":0}

    def start_driver(self):
        options = ChromeOptions()
        options.headless = True
        options.add_argument("--log-level=3")
        self.driver = webdriver.Chrome(options=options)
        self.driver.get(self.url)
        self.page = BeautifulSoup(self.driver.page_source, "html.parser")

    def test_page(self):
        """This function executes the tests for the current page. If tests for subpages are enabled, it will also test all subpages"""
        self.page = BeautifulSoup(self.driver.page_source, "html.parser")
        print("\n\n" + self.driver.current_url + "\n---------------------")
        self.check_doc_language()
        self.check_alt_texts()
        self.check_input_labels()
        self.check_buttons()
        self.check_links()
        self.check_color_contrast()

    def check_doc_language(self):
        """This function checks if the doc language is set (3.1.1 H57)"""
        # check if language attribute exists and is not empty
        lang_attr = self.page.find("html").get_attribute_list("lang")[0]
        if not lang_attr is None and not lang_attr == "":
            print("  Document language is set")
            self.correct["doc_language"] += 1
        elif not lang_attr is None:
            print("x Document language is empty")
            self.wrong["doc_language"] += 1
        else:
            print("x Document language is missing")
            self.wrong["doc_language"] += 1

    def check_alt_texts(self):
        """This function checks if all images on the page have an alternative text (1.1.1 H37)"""
        # get all img elements
        img_elements = self.page.find_all("img")
        for img_element in img_elements:
            # check if img element has an alternative text that is not empty
            alt_text = img_element.get_attribute_list('alt')[0]
            if not alt_text is None and not alt_text == "":
                print("  Alt text is correct", Utils.xpath_soup(img_element))
                self.correct["alt_texts"] += 1
            elif not alt_text is None:
                print("x Alt text is empty", Utils.xpath_soup(img_element))
                self.wrong["alt_texts"] += 1
            else:
                print("x Alt text is missing", Utils.xpath_soup(img_element))
                self.wrong["alt_texts"] += 1

    def check_input_labels(self):
        """This function checks if all input elements on the page have some form of label (1.3.1 H44 & ARIA16)"""
        # get all input and label elements
        input_elements = self.page.find_all("input")
        label_elements = self.page.find_all("label")
        for input_element in input_elements:
            # exclude input element of type hidden, submit, reset and button
            if ("type" in input_element.attrs and not input_element['type'] == "hidden" and not input_element['type'] == "submit" \
                    and not input_element['type'] == "reset" and not input_element['type'] == "button") or "type" not in input_element.attrs:
                # check if input is of type image and has a alt text that is not empty
                if "type" in input_element.attrs and input_element['type'] == "image" and "alt" in input_element.attrs \
                        and not input_element['alt'] == "":
                    print("  Input of type image labelled with alt text", Utils.xpath_soup(input_element))
                    self.correct["input_labels"] += 1
                # check if input element uses aria-label
                elif "aria-label" in input_element.attrs and not input_element['aria-label'] == "":
                    print("  Input labelled with aria-label attribute", Utils.xpath_soup(input_element))
                    self.correct["input_labels"] += 1
                # check if input element uses aria-labelledby
                elif "aria-labelledby" in input_element.attrs and not input_element['aria-labelledby'] == "":
                    label_element = self.page.find(id=input_element['aria-labelledby'])
                    if not label_element is None:
                        texts_in_label_element = label_element.findAll(string=True)
                        if not texts_in_label_element == []:
                            print("  Input labelled with aria-labelledby attribute", Utils.xpath_soup(input_element))
                            self.correct["input_labels"] += 1
                        else:
                            print("x Input labelled with aria-labelledby attribute, but related label has no text", Utils.xpath_soup(input_element))
                            self.wrong["input_labels"] += 1
                    else:
                        print("x Input labelled with aria-labelledby attribute, but related label does not exist", Utils.xpath_soup(input_element))
                        self.wrong["input_labels"] += 1
                else:
                    # check if input element has a corresponding label element
                    label_correct = False
                    for label_element in label_elements:
                        # check if "for" attribute of label element is identical to "id" of input element
                        if "for" in label_element.attrs and "id" in input_element.attrs and label_element['for'] == input_element['id']:
                            label_correct = True
                    if label_correct:
                        print("  Input labelled with label element", Utils.xpath_soup(input_element))
                        self.correct["input_labels"] += 1
                    else:
                        print("x Input not labelled at all", Utils.xpath_soup(input_element))
                        self.wrong["input_labels"] += 1

    def check_buttons(self):
        """This function checks if all buttons and input elements of the types submit, button and reset have some form of content (1.1.1 & 2.4.4)"""
        # get all buttons and input elements of the types submit, button and reset
        input_elements = self.page.find_all("input", type=["submit", "button", "reset"])
        button_elements = self.page.find_all("button")

        for input_element in input_elements:
            # check if input element has a value attribute that is not empty
            if "value" in input_element.attrs and not input_element['value'] == "":
                print("  Button has content", Utils.xpath_soup(input_element))
                self.correct["empty_buttons"] += 1
            else:
                print("x Button is empty", Utils.xpath_soup(input_element))
                self.wrong["empty_buttons"] += 1

        for button_element in button_elements:
            # check if the button has content or a title
            texts = button_element.findAll(string=True)
            if not texts == [] or ("title" in button_element.attrs and not button_element["title"] == ""):
                print("  Button has content", Utils.xpath_soup(button_element))
                self.correct["empty_buttons"] += 1
            else:
                print("x Button is empty", Utils.xpath_soup(button_element))
                self.wrong["empty_buttons"] += 1

    def check_links(self):
        """This function checks if all links on the page have some form of content (2.4.4 G91 & H30)"""
        # get all a elements
        link_elements = self.page.find_all("a")
        for link_element in link_elements:
            # check if link has content
            texts_in_link_element = link_element.findAll(string=True)
            img_elements = link_element.findChildren("img", recursive=False)
            all_alt_texts_set = True
            for img_element in img_elements:
                alt_text = img_element.get_attribute_list('alt')[0]
                if alt_text is None or alt_text == "":
                    all_alt_texts_set = False
            if not texts_in_link_element == [] or (not img_elements == [] and all_alt_texts_set):
                print("  Link has content", Utils.xpath_soup(link_element))
                self.correct["empty_links"] += 1
            else:
                print("x Link is empty", Utils.xpath_soup(link_element))
                self.wrong["empty_links"] += 1

    def check_color_contrast(self):
        """This function checks if all texts on the page have high enough contrast to the color of the background (1.4.3 G18 & G145 (& 148))"""
        # exclude script, style, title and empty elements as well as doctype and comments
        texts_on_page = Utils.extract_texts(self.page)
        input_elements = self.page.find_all("input")
        elements_with_text = texts_on_page + input_elements
        for text in elements_with_text:
            selenium_element = self.driver.find_element(by="xpath", value=Utils.xpath_soup(text))
            # exclude invisible texts
            element_visible = selenium_element.value_of_css_property('display')
            if not element_visible == "none" and (not text.name == "input" or (text.name == "input" \
                    and "type" in text.attrs and not text['type'] == "hidden")):
                text_color = Utils.convert_to_rgba_value(selenium_element.value_of_css_property('color'))
                background_color = Utils.get_background_color(self.driver, text)

                # calculate contrast between text color and background color
                contrast = Utils.get_contrast_ratio(eval(text_color[4:]), eval(background_color[4:]))

                # get font size and font weight
                font_size = selenium_element.value_of_css_property('font-size')
                font_weight = selenium_element.value_of_css_property('font-weight')


                if not font_size is None and font_size.__contains__("px") and \
                        (int(''.join(filter(str.isdigit, font_size))) >= 18 or ((font_weight == "bold" or font_weight == "700" \
                        or font_weight == "800" or font_weight == "900" or text.name == "strong") \
                        and int(''.join(filter(str.isdigit, font_size))) >= 14)):
                    if contrast >= 3:
                        print("  Contrast meets minimum requirements", Utils.xpath_soup(text), text_color, background_color)
                        self.correct["color_contrast"] += 1
                    else:
                        print("x Contrast does not meet minimum requirements", Utils.xpath_soup(text), text_color, background_color)
                        self.wrong["color_contrast"] += 1
                else:
                    if contrast >= 4.5:
                        print("  Contrast meets minimum requirements", Utils.xpath_soup(text), text_color, background_color)
                        self.correct["color_contrast"] += 1
                    else:
                        print("x Contrast does not meet minimum requirements", Utils.xpath_soup(text), text_color, background_color)
                        self.wrong["color_contrast"] += 1

    def calculate_result(self):
        """This function calculates the result of the test and prints it to the console"""
        # calculate correct and false implementations
        correct = sum(self.correct.values())
        false = sum(self.wrong.values())
        if correct == 0 and false == 0:
            print("Nothing found")
            return
        print("\nResult")
        print("---------------------")
        print("Correct:", correct)
        for category, value in self.correct.items():
            print(" ", category + ":", value)
        print("Errors:", false)
        for category, value in self.wrong.items():
            print(" ", category + ":", value)
        print("Ratio (correct to total):", round(correct/(correct+false), 2), "\n")

        # check if ratio correct/total reaches wanted minimum value
        if correct/(correct+false) >= self.required_degree:
            print("Accessibility test successful - can deploy")
        else:
            sys.tracebacklimit = 0
            raise Exception("Too many accessibility errors - try fix them!")


at = AccessibilityTester('https://www.uno.edu.ar')
at.start_driver()
at.test_page()
at.driver.quit()
at.calculate_result()



https://www.uno.edu.ar/
---------------------
  Document language is set
  Alt text is correct /html/body/div[1]/div[2]/div/div/div/div/div/div/a[1]/img
  Alt text is correct /html/body/div[1]/div[2]/div/div/div/div/div/div/a[2]/img
  Alt text is correct /html/body/div[1]/div[2]/div/div/div/div/div/div/div/ul/li[1]/a/img
  Alt text is correct /html/body/div[1]/div[2]/div/div/div/div/div/div/div/ul/li[2]/a/img
  Alt text is correct /html/body/div[1]/div[2]/div/div/div/div/div/div/div/ul/li[3]/a/img
  Alt text is correct /html/body/div[1]/div[3]/div[1]/div/div/div/div/div/div[1]/img
  Alt text is correct /html/body/div[1]/div[3]/div[3]/div/div/div/div/div/div/article[1]/img
  Alt text is correct /html/body/div[1]/div[3]/div[3]/div/div/div/div/div/div/article[2]/img
  Alt text is correct /html/body/div[1]/div[3]/div[3]/div/div/div/div/div/div/article[3]/img
  Alt text is correct /html/body/div[1]/div[3]/div[3]/div/div/div/div/div/div/article[4]/img
  Alt text is correct /html/body/div[1